# Week3 assignment

#### Import pandas

In [1]:
import pandas as pd

#### import HTML data by pandas read_html method.

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(data[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### ignore cells without Borough.

In [3]:
df = df[df['Borough'] != 'Not assigned'].reset_index()
df = df[['Postcode','Borough','Neighbourhood']]
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### check the value of Neighbourhood.
#### if it is 'Not assigned' add Borough.

In [4]:
for i in range(len(df)):
    if df.Neighbourhood[i] == 'Not assigned':
        df.Neighbourhood[i] = df.Borough[i]
        print(df.Neighbourhood[i])

Queen's Park


#### combine in Neighbours with the same Postcode to one row with the neighborhoods separated with a comma.

In [5]:
df2 = df.groupby('Postcode').Neighbourhood.apply(lambda s : ','.join(s))
df2 = pd.DataFrame(df2)
df2.head(10)

,Neighbourhood
Postcode,
M1B,"Rouge,Malvern"
M1C,"Highland Creek,Rouge Hill,Port Union"
M1E,"Guildwood,Morningside,West Hill"
M1G,Woburn
M1H,Cedarbrae
M1J,Scarborough Village
M1K,"East Birchmount Park,Ionview,Kennedy Park"
M1L,"Clairlea,Golden Mile,Oakridge"
M1M,"Cliffcrest,Cliffside,Scarborough Village West"


In [6]:
df3 = df[['Postcode','Borough']]
df3.head()

,Postcode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M5A,Downtown Toronto
4,M6A,North York


In [7]:
df = pd.merge(df3, df2, on='Postcode').drop_duplicates()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


#### check the shape of dataframe.

In [8]:
df.shape

(103, 3)

#### install geocoder

In [9]:
!pip install geocoder

#### import module and define function to get latitudes and longitudes.

In [10]:
import geocoder # import geocoder

def geo(postal_code, lat):
    lat_lng_coords  = lat
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    data = [lat_lng_coords[0], lat_lng_coords[1]]
    
    return data

#### insert rows.

In [11]:
#df['latitude'] = ''
#df['longitude'] = ''
#df.head()

#### run the function for all Post code.

In [ ]:
for i in range(len(df)):
    # initialize your variable to Non
    lat = None
    geo(df.Postcode[i], lat)
    df.latitude[i] = data[0]
    df.longitude[i] = data[1]
    print(i)
df.head()

In [ ]:
df_lola = pd.read_csv('https://cocl.us/Geospatial_data')
df_new = df_lola.rename(columns={'Postal Code': 'Postcode'})
df_new.head()

In [ ]:
df = pd.merge(df, df_new, on='Postcode').drop_duplicates()
df.head()

#### K-means clustering

#### import libraty.

In [ ]:
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [ ]:
df.dtypes

#### choose items including 'Toronto'.

In [ ]:
df_tor = df[df['Borough'].str.contains('Toronto')]
df_tor.head()

#### Plot map and cluster.

In [ ]:
address  =  'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map